# Roteiro - AEDE (Análise exploratória de dados espaciais) global

In [0]:
# instalação dos pacotes necessários
!sudo apt install python3-rtree   # solução para o problema
!pip install geopandas pysal mapclassify

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gp
import pysal as ps
import seaborn as sns
from pysal.lib.weights.contiguity import Queen
from pysal.explore.esda import Moran, Moran_Local
from pysal.viz.splot.esda import plot_moran, moran_scatterplot, lisa_cluster, plot_local_autocorrelation
from pysal.viz import mapclassify
link = 'https://raw.githubusercontent.com/lincolnfrias/dados/master/'
link_p = 'https://raw.githubusercontent.com/patriciasiqueira/patriciasiqueira.github.io/master/arquivos/'

In [0]:
# ler dados de interesse
dados = pd.read_csv(link_p+'qtde-22.csv', encoding='latin1')
# quantidade produzida em toneladas em 2017

1. Complete o que estiver faltando nas células abaixo:

In [0]:
# nomes das variáveis


In [0]:
# número de linhas e colunas do dataframe


In [0]:
# mostrar algumas linhas do dataframe


In [0]:
# renomear colunas
# 'ibge7' será 'mun' (código do município)
# 'mun' será 'município' (nome do município)
dados.rename(columns={'mun': 'municipio'}, inplace=True)  # inplace = True modifica o dataframe
dados.rename(columns={'ibge7': 'mun'}, inplace=True)

In [0]:
# ler shapefile
link = 'https://raw.githubusercontent.com/lincolnfrias/dados/master/mg.json'
geodf = gp.read_file(link)

In [0]:
# visualizar as primeiras linhas do shapefile


In [0]:
# mesclar shapefile com dataframe
# usando a coluna mun
geodf.rename(columns={'CD_GEOCMU': 'mun'}, inplace=True)  # mudar nome da coluna com cód. para 'mun'
geodf['mun'] = geodf.mun.astype(int)  # transformar códigos em inteiros
mg = pd.merge(geodf, dados, on='mun', suffixes=('', '_y'))  # mesclar o dataframe e o shapefile
mg.shape

In [0]:
# visualizar primeiras linhas objeto resultante (dataframe + shapefile)


In [0]:
# colunas (variáveis) do objeto resultante
mg.columns

In [0]:
# resumo estátistico de todas as variáveis - apenas considerando as culturas (coluna 5 em diante)
mg.iloc[:, 5:].describe()

In [0]:
# escolher uma variável dos dados

# variavel = 'batata-inglesa'

In [0]:
# apenas plotar o mapa de Minas Gerais usando o dataframe criado
# mapa temático - simples


**Mapas temáticos**

Mapas temáticos possibilitam visualizar a distribuição espacial de alguma variável por categorias.

- PuBu

- GnBu

- Oranges

- BuGn

- Purples

- YlOrBr


cores: cmap (https://matplotlib.org/tutorials/colors/colormaps.html)


2. Fazer a análise de cada mapa temático obtido e chamar a atenção para a diferença entre os diferentes esquemas utilizados:

- a) quantis
- b) intervalos iguais
- c) fisher jenks

Alterar esquemas, cores e tamanhos nos mapas temáticos a seguir:

In [0]:
# mapa temático - com opções
# scheme='quantiles'
# atribui mesma quantidade de valores para cada categoria. 
# apesar de esconder valores extremos, pode se tornar mais informativa se a distribuição for assimétrica)
mg.plot(column=variavel, figsize=(10, 10), scheme='quantiles', cmap='OrRd', legend=True, k=4);

Interpretação:

- 
- 
- 

In [0]:
# usar scheme='equal_interval'



Interpretação:

- 
- 
- 

In [0]:
# usar scheme='fisher_jenks'


Interpretação:

- 
- 
- 

In [0]:
# obter matriz de vizinhança no formato queen
w = Queen.from_dataframe(mg)
w.transform = 'r'

### I de Moran

Estatística mais utilizada para medir a autocorrelação espacial. Ela mede a relação do desvio padronizado de uma variável numa área com o desvio padronizado das áreas vizinhas para a mesma variável:

$$I=\frac{N}{S_{0}}\frac{\displaystyle\sum_{i=1}^{n}\sum_{j=1}^{n}w_{ij}z_{i}z_{j}}{\displaystyle\sum_{i=1}^{n}z^{2}_{i}},$$
em que:

- $z_i = y_i - \bar{y}$ representa o desvio em relação à média da variável.
- $y_i$: valor da variável em um determinado local $i$
- $N$: número de observações
- $S_0 = \sum\sum w_{ij}$
- $E[I] = -1/(N - 1) \approx 0$
            
Se o valor-$p$ referente ao teste do $I$ de Moran for significativo, podemos olhar para o valor da estatística $I$ e concluir:

- $I > 0$: autocorrelação espacial positiva (*clusters* espaciais - HH, LL)
- $I < 0$: autocorrelação espacial negativa (*outliers* espaciais - HL, LH)

In [0]:
# calcular I de Moran global para a variável escolhida
y = mg[variavel].values
moran = Moran(y, w)
moran.I

Ver opções possíveis para os valores-$p$:

digitar moran. e depois TAB

In [0]:
# valor-p
moran.p_sim

4 - Qual a interpretação para o $I$ de Moran global?

- 
- 
- 

In [0]:
# diagrama de dispersão de Moran
plot_moran(moran, zstandard=False, figsize=(10,4));

In [0]:
# só o diagrama de dispersão
moran_loc = Moran_Local(y, w)
moran_scatterplot(moran_loc, p=0.05);